In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3

In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:.7f}  [{current:5d} / {size:5d}]')


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)

            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f'Test error:\nAccuracy: {(100 * correct):.1f}%, Avg loss: {test_loss:.8f}')

In [6]:
epochs = 10

for t in range(epochs):
    print(f'Epoch: {t + 1}\n----------------------------------------')

    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print('Done')

Epoch: 1
----------------------------------------
Loss: 2.2918308  [    0 / 60000]
Loss: 2.2904694  [ 6400 / 60000]
Loss: 2.2657256  [12800 / 60000]
Loss: 2.2737236  [19200 / 60000]
Loss: 2.2476225  [25600 / 60000]
Loss: 2.2138462  [32000 / 60000]
Loss: 2.2280505  [38400 / 60000]
Loss: 2.1836836  [44800 / 60000]
Loss: 2.1711059  [51200 / 60000]
Loss: 2.1640198  [57600 / 60000]
Test error:
Accuracy: 38.8%, Avg loss: 2.14984858
Epoch: 2
----------------------------------------
Loss: 2.1490889  [    0 / 60000]
Loss: 2.1499562  [ 6400 / 60000]
Loss: 2.0876100  [12800 / 60000]
Loss: 2.1167309  [19200 / 60000]
Loss: 2.0574424  [25600 / 60000]
Loss: 2.0015385  [32000 / 60000]
Loss: 2.0292854  [38400 / 60000]
Loss: 1.9474146  [44800 / 60000]
Loss: 1.9342191  [51200 / 60000]
Loss: 1.8986399  [57600 / 60000]
Test error:
Accuracy: 55.5%, Avg loss: 1.88112001
Epoch: 3
----------------------------------------
Loss: 1.9020448  [    0 / 60000]
Loss: 1.8853025  [ 6400 / 60000]
Loss: 1.7608511  [12800 